In [2]:
!pip install torchaudio
!pip install librosa matplotlib tensorflow_io

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/six-1.16.0.dist-info/METADATA'

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
import itertools
import random
from typing import Tuple
import wave
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import random_split
import torchaudio.transforms as T
from IPython.display import Audio
from torch.utils.data import Dataset, DataLoader, Subset
import math, random
import torchaudio
from torchaudio import transforms
from IPython.display import Audio
import pdb
import pandas as pd
import numpy as np

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/torchaudio/_internal/module_utils.py:87: UserWarning: Failed to import soundfile. 'soundfile' backend is not available.
  warnings.warn("Failed to import soundfile. 'soundfile' backend is not available.")


In [164]:
df = pd.read_pickle("df_final.pkl")
df

,audio,intensity
0,"[[tensor(-0.0644), tensor(-0.0727), tensor(-0....",0.000000
1,"[[tensor(-0.7592), tensor(-0.8084), tensor(-0....",95.076299
2,"[[tensor(-0.0686), tensor(-0.0764), tensor(-0....",70.015840
3,"[[tensor(-1.0000), tensor(-1.0000), tensor(-1....",67.107005
4,"[[tensor(-0.0646), tensor(-0.0725), tensor(-0....",4.039027
...,...,...
9730,"[[tensor(0.2384), tensor(0.2235), tensor(0.209...",0.000000
9731,"[[tensor(0.5547), tensor(0.5233), tensor(0.493...",18.190045
9732,"[[tensor(0.2384), tensor(0.2235), tensor(0.209...",0.611846
9733,"[[tensor(0.5199), tensor(0.4874), tensor(0.456...",2.546859


In [165]:
sample_rate = 48000
df.loc[0, "audio"].shape

torch.Size([2, 192000])

In [166]:
bins = [-1, 20, 40, 60, 80, 100]
labels = [5,4,3,2,1]
df['rating'] = pd.cut(df['intensity'], bins,labels=labels)

In [167]:
df.head()

,audio,intensity,rating
0,"[[tensor(-0.0644), tensor(-0.0727), tensor(-0....",0.000000,5
1,"[[tensor(-0.7592), tensor(-0.8084), tensor(-0....",95.076299,1
2,"[[tensor(-0.0686), tensor(-0.0764), tensor(-0....",70.015840,2
3,"[[tensor(-1.0000), tensor(-1.0000), tensor(-1....",67.107005,2
4,"[[tensor(-0.0646), tensor(-0.0725), tensor(-0....",4.039027,5


In [168]:
def toStereo(aud):
    if (aud.shape[0] == 2):
        return aud
    resig = torch.cat([aud, aud])

    return resig

In [169]:
def rezise(aud):
    num_rows, sig_len = aud.shape
    max_len = 48000//1000 * 4000

    if (sig_len > max_len):
        aud = aud[:,:max_len]

    elif (sig_len < max_len):
        pad_begin_len = random.randint(0, max_len - sig_len)
        pad_end_len = max_len - sig_len - pad_begin_len

        pad_begin = torch.zeros((num_rows, pad_begin_len))
        pad_end = torch.zeros((num_rows, pad_end_len))

        aud = torch.cat((pad_begin, aud, pad_end), 1)
      
    return aud

In [170]:
class TransformedData(Dataset):
    """Transform the first channel of audio and return it together with the rating."""
    
    def __init__(self, df, transforms: nn.Sequential):
        self.df = df
        self.transforms = transforms
    
    def __len__(self) -> int:
        return self.df.shape[0]
    
    def __getitem__(self, n: int) -> Tuple[torch.Tensor, torch.Tensor]:
        example = self.df.loc[n]
        channel = example["audio"]
        channel_str = toStereo(channel)
        channel_rezise = rezise(channel_str)
        return self.transforms(channel_rezise), int(example["rating"])

In [171]:
reduced_sample_rate = 16000
window_size = 2048
hop_size = 256
num_mels = 128

In [172]:
transforms = nn.Sequential(
    T.Resample(orig_freq=sample_rate, new_freq=reduced_sample_rate),
    T.MelSpectrogram(
        n_mels=num_mels,
        n_fft=window_size,
        win_length=window_size,
        hop_length=hop_size,
        power=1,
        center=False,
        sample_rate=reduced_sample_rate,
        f_min=0,
        f_max=reduced_sample_rate / 2,
        window_fn=torch.hann_window
    ),
)

In [173]:
from torch.utils.data import random_split

myds = TransformedData(df, transforms)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])

# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, shuffle=False)
myds[3]

(tensor([[[ 21.4689,  18.8361,  29.7160,  ...,  41.7658,  48.4239,  47.4438],
          [ 19.4787,  22.3880,  26.1348,  ..., 120.7119, 123.5909, 118.5946],
          [ 16.3506,   8.8245,  11.7256,  ..., 202.9790, 181.3675, 169.8651],
          ...,
          [  3.6043,   3.1011,   2.9115,  ...,   1.7462,   2.0075,   2.5565],
          [  4.0006,   3.6196,   3.5373,  ...,   1.4426,   1.5033,   1.8783],
          [  3.3384,   2.9602,   2.7348,  ...,   1.4244,   1.7925,   2.2975]],
 
         [[ 18.3951,  12.1004,  12.5335,  ...,  21.2163,  29.3659,  33.0858],
          [ 28.3296,  26.9679,  23.2514,  ...,  15.9717,  20.3360,  37.7313],
          [ 15.4397,  15.5327,  16.5888,  ..., 136.4331, 146.8740, 141.9935],
          ...,
          [  4.4524,   3.9819,   3.7549,  ...,   2.0857,   2.0925,   2.2385],
          [  5.0306,   4.3849,   4.0496,  ...,   2.1031,   2.1256,   2.3107],
          [  6.6391,   6.3637,   5.0606,  ...,   2.3279,   2.8345,   3.1140]]]),
 2)

In [174]:
import torch.nn.functional as F
from torch.nn import init

import torch.nn.functional as F
from torch.nn import init

# ----------------------------
# Audio Classification Model
# ----------------------------
class CNN1(torch.nn.Module):
    def __init__(self):
        super(CNN1, self).__init__()
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

        self.bn1 = nn.BatchNorm2d(8)
        self.bn2 = nn.BatchNorm2d(16)
        self.bn3 = nn.BatchNorm2d(32)
        self.bn4 = nn.BatchNorm2d(64)

        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p=0.2)

        init.kaiming_normal_(self.conv4.weight, a=0.1)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        init.kaiming_normal_(self.conv2.weight, a=0.1)

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=6)

        self.conv1.bias.data.zero_()
        self.conv2.bias.data.zero_()
        self.conv3.bias.data.zero_()
        self.conv4.bias.data.zero_()

    def forward(self, x):
        # TODO: Complete the forward pass
        x = self.relu(self.conv1(x))
        x = self.bn1(x)
        x = self.relu(self.conv2(x))
        x = self.bn2(x)
        x = self.relu(self.conv3(x))
        x = self.bn3(x)
        x = self.relu(self.conv4(x))
        x = self.bn4(x)
        x = self.drop(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        return x



In [175]:
epochs = 10
model = CNN1()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
train_loss = []
# traing iteration
for epoch in range(epochs):
    running_loss = 0
    for itr, (audio, label) in enumerate(train_dl):
        # zero gradient
        optimizer.zero_grad()
        # forward path
        y_predicted = model(audio)
        loss = criterion(y_predicted, label)
        running_loss += loss.item()
        # backpropagating
        loss.backward()
        # optimizes the weights
        optimizer.step()
    train_loss.append(running_loss)
    print(f'epoch: {epoch+1}, loss: {running_loss:.4f}')

epoch: 1, loss: 363.2806
epoch: 2, loss: 331.9980
epoch: 3, loss: 325.3421
epoch: 4, loss: 322.5406
epoch: 5, loss: 319.8052
epoch: 6, loss: 317.9562
epoch: 7, loss: 315.5428
epoch: 8, loss: 314.2644
epoch: 9, loss: 311.3648
epoch: 10, loss: 312.0581


In [176]:
from scipy import stats
correct = 0
total = 0
array_pred = torch.empty(1)
array_correct = torch.empty(1)
with torch.no_grad():
    for itr, (audio, label) in enumerate(val_dl):
        outputs = model(audio)
        _, predicted = torch.max(outputs.data, 1)
        correct += predicted.eq(label.reshape(len(label),)).sum() 
        total += float(len(label))
        array_pred = torch.cat((array_pred, predicted), 0)
        array_correct = torch.cat((array_correct, label.reshape(len(label),)))
    accuracy = correct / total
    print(f'Accuracy of Neural Network is {accuracy:.4f}')
stats.spearmanr(array_correct, array_pred)

Accuracy of Neural Network is 0.4551


SpearmanrResult(correlation=0.4328661999857933, pvalue=8.792357338765065e-90)